# **GT4Py Tutorial : Stencil Basics**

## **Introduction**

This notebook will show how to create a simple GT4Py stencil that copies data from one variable to another.

### **Notebook Requirements**

- Python v3.11.x to v3.12.x
- [NOAA/NASA Domain Specific Language Middleware](https://github.com/NOAA-GFDL/NDSL)
- `ipykernel==6.1.0` (via `pip install ipykernel==6.1.0`)
- [`ipython_genutils`](https://pypi.org/project/ipython_genutils/)

### **Quick GT4Py (Cartesian version) Overview**

GT4Py is a Domain Specific Language (DSL) in Python that enables a developer to write stencil computations.  Compared to simply running under Python, GT4Py achieves performance when the Python code is transformed and compiled into a lower level language such as C++ and CUDA. This code transformation capability also enables a GT4Py codebase the capability to execute on multiple architectures.  In this notebook, we will cover the basics of creating GT4Py stencils and demonstrate several intracies of the DSL. Additional information about GT4Py can be found at the [GT4Py site](https://gridtools.github.io/gt4py/latest/index.html).  One small note is that this tutorial covers and uses the Cartesian version of GT4Py and not the unstructured version.

### **GT4Py Parallel/Execution Model**

Within a 3-dimensional domain, GT4Py considers computations in two parts.  If we assume an `(I,J,K)` coordinate system as a reference, GT4Py separates computations in the Horizontal (`IJ`) spatial plane and Vertical (`K`) spatial dimension.  In the Horizontal spatial plane, computations are implicitly executed in parallel, which also means that there is no assumed calculation order within the plane.  In the Vertical spatial dimension, comptuations are specified by an iteration policy that will be discussed later through examples.

Another quick note is that the stencil computations are executed sequentially in the order they appear in code.

## **Tutorial**

### **Copy Stencil example**

To demonstrate how to implement a GT4Py stencil, we'll step through a code example that copies the values of one array into another array.  First, we import several packages. 

In [ ]:
from ndsl.dsl.gt4py import PARALLEL, computation, interval, stencil
from ndsl.dsl.typing import FloatField
from ndsl.quantity import Quantity
import numpy as np

As we walk through the example, we'll highlight different terms and such from the imported packages.  Let's first define, in GT4Py terms, two arrays of size 5 by 5 by 2 (dimensionally `I` by `J` by `K`).  These arrays are defined using a `Quantity` object, an NDSL data container for physical quantities.  More detailed information about the `Quantity` object and its arguments can be found from the [`Quantity` docstring](https://github.com/NOAA-GFDL/NDSL/blob/develop/ndsl/quantity.py#L270).  To make debugging easier, the `numpy` backend will be used.

In [ ]:
backend = "numpy"

nx = 5
ny = 5
nz = 2

shape = (nx, ny, nz)

qty_out = Quantity(
    data=np.zeros([nx, ny, nz]), dims=["I", "J", "K"], units="m", gt4py_backend=backend
)

arr = np.indices(shape, dtype=float).sum(
    axis=0
)  # Value of each entry is sum of the I and J index at each point

qty_in = Quantity(data=arr, dims=["I", "J", "K"], units="m", gt4py_backend=backend)

We will next create a simple GT4Py stencil that copies values from one input to another.  A stencil will look like a Python subroutine or function except that it uses specific GT4Py functionalities.

In [ ]:
@stencil(backend=backend)
def copy_stencil(input_field: FloatField, output_field: FloatField):
    with computation(PARALLEL), interval(...):
        output_field = input_field

As mentioned before, GT4Py (cartesian version) was designed for stencil-based computation. Since stencil calculations generally are localized computations, GT4Py stencils are written using variables and the variable's **relative location** if the variable is an array.  This means that for GT4Py stencil computations, all variable locations on the right-hand side of the `=` are relative to the variable on the left-hand side of the `=`.  For example, a stencil-based [Laplacian](https://en.wikipedia.org/wiki/Discrete_Laplace_operator#Finite_differences) calculation can be implemented as follows.

In [4]:
def laplacian(input_field: FloatField, output_field: FloatField):
    with computation(PARALLEL), interval(...):
        output_field = (
            -4.0 * input_field[0, 0, 0]
            + input_field[1, 0, 0]
            + input_field[-1, 0, 0]
            + input_field[0, 1, 0]
            + input_field[0, -1, 0]
        )

In the `laplacian` stencil, every value of `output_field` at a particular location is a function of `input_field` at that same location (`input_field[0,0,0]`), the adjacent `input_field` values in the `I`-direction (`input_field[1,0,0]` and `input_field[-1,0,0]`), and the adjacent `input_field` values in the `J`-direction (`input_field[0,1,0]` and `input_field[0,-1,0]`)

If there are no indices in brackets next to a GT4Py type (such as `FloatField`), it's implied to be at the `[0]` (for 1-dimension), `[0,0]` (for 2-dimension), or `[0,0,0]` (for 3-dimension) location.  Note again that these locations are **relative**.  In the `laplacian` stencil, `output_field` has an implied `[0,0,0]` location, and `input_field[0,0,0]` could have been written simply as `input_field`.

We see that both `copy_stencil` and the `laplacian` stencil do not contain any explicit loops.  As mentioned above in the notebook, GT4Py has a particular computation policy that implicitly executes in parallel within an `IJ` plane and is user defined in the `K` dimension.  This execution policy in the `K` dimension is dictated by the `computation` and `interval` keywords.  
- `with computation(PARALLEL)` means that there's no order preference to executing the `K` interval.  This also means that the `K` interval can be computed in parallel to potentially gain performance if computational resources are available.

- `interval(...)` means that the "entire" `K` dimension range is executed (as dictated by the keywords `origin` and `domain`, which will be covered [later in the notebook](#setting-domain-subsets-in-a-stencil-call)).  Instead of `(...)`, more specific intervals can be specified using a tuple of two integers.  For example... 

    - `interval(0,2)` : The interval `K` = 0 to 1 is executed.
    - `interval(0,-1)` : The interval `K` = 0 to N-2 (where N is the size of `K`) is executed.
    - Note : The actual intervals for the above two `interval` are also dictated by [`origin` and `domain`](#setting-domain-subsets-in-a-stencil-call)

The decorator `@stencil(backend=backend)` (Note: `stencil` comes from the package `ndsl.dsl.gt4py`) converts `copy_stencil` to use the specified `backend` to "compile" the stencil.  `stencil` can also be a function call to create a stencil object.

In [ ]:
copy_stencil_numpy = stencil(backend="numpy", definition=copy_stencil)

Note that the input and output parameters to `copy_stencil` and `laplacian` are of type `FloatField`, which can essentially be thought of as a 3-dimensional NumPy array of `float` types.

`plot_field_at_kN` plots the values within the `IJ` plane at `K = 0` if no integer is specified or at `K` equal to the integer that is specified as an argument.  As we can see in the plots below, `copy_stencil` copies the values from `qty_in` into `qty_out`.

In [ ]:
print("Plotting values of qty_in at K = 0")
qty_in.plot_k_level(0)
print("Plotting values of qty_out at K = 0")
qty_out.plot_k_level(0)
print("Executing `copy_stencil`")
copy_stencil(qty_in, qty_out)
print("Plotting qty_out from `copy_stencil` at K = 0")
qty_out.plot_k_level(0)
print("Plotting qty_out from `copy_stencil` at K = 1")
qty_out.plot_k_level(1)

### **Setting domain subsets in a stencil call**

GT4Py also allows a subset to be specified from a stencil call and executed in a fashion similar to using `interval(...)` in the `K` dimension.  This is done by setting the stencil call's `origin` and `domain` argument.

- `origin` : This specifies the "starting" coordinate to perform computations.  

- `domain` : This specifies the range of the stencil computation based on `origin` as the "starting" coordinate 

- Note: May need to check whether `domain` and `origin` affect `interval()`

If these two parameters are not set, the stencil call by default will iterate over the entire input domain.  The following demonstrates the effect of specifying different `origin` and `domain`.

In [ ]:
qty_out = Quantity(
    data=np.zeros([nx, ny, nz]), dims=["I", "J", "K"], units="m", gt4py_backend=backend
)

print("Plotting values of qty_in at K = 0")
qty_in.plot_k_level(0)
print("Plotting values of qty_out at K = 0")
qty_out.plot_k_level(0)
print("Executing `copy_stencil` with origin=(1, 0, 0)")
copy_stencil(qty_in, qty_out, origin=(1, 0, 0))
print("Plotting qty_out at K = 0 based on `copy_stencil` with origin=(1, 0, 0)")
qty_out.plot_k_level(0)

qty_out = Quantity(
    data=np.zeros([nx, ny, nz]), dims=["I", "J", "K"], units="m", gt4py_backend=backend
)

print("Resetting qty_out to zero...")
print("Plotting values of qty_out at K = 0")
qty_out.plot_k_level(0)
print("Executing `copy_stencil` with origin=(0, 1, 0)")
copy_stencil(qty_in, qty_out, origin=(0, 1, 0))
print("Plotting qty_out at K = 0 based on `copy_stencil` with origin=(0, 1, 0)")
qty_out.plot_k_level(0)

qty_out = Quantity(
    data=np.zeros([nx, ny, nz]), dims=["I", "J", "K"], units="m", gt4py_backend=backend
)

print("Resetting qty_out to zero...")
print("Plotting values of qty_out at K = 0")
qty_out.plot_k_level(0)
print("Executing `copy_stencil` with origin = (0, 0, 1)")
copy_stencil(qty_in, qty_out, origin=(0, 0, 1))
print("Plotting qty_out at K = 0 based on `copy_stencil` with origin=(0, 0, 1)")
qty_out.plot_k_level(0)
print("Plotting qty_out at K = 1 based on `copy_stencil` with origin=(0, 0, 1)")
qty_out.plot_k_level(1)

qty_out = Quantity(
    data=np.zeros([nx, ny, nz]), dims=["I", "J", "K"], units="m", gt4py_backend=backend
)
print("Resetting qty_out to zero...")
print("Plotting values of qty_in at K = 0")
qty_in.plot_k_level(0)
print("Plotting values of qty_out at K = 0")
qty_out.plot_k_level(0)
print("Executing `copy_stencil` with domain=(2, 2, nz)")
copy_stencil(qty_in, qty_out, domain=(2, 2, nz))
print("Plotting qty_out at K = 0 based on `copy_stencil` with domain=(2, 2, nz)")
qty_out.plot_k_level(0)

qty_out = Quantity(
    data=np.zeros([nx, ny, nz]), dims=["I", "J", "K"], units="m", gt4py_backend=backend
)
print("Resetting qty_out to zero...")
print("Plotting values of qty_out at K = 0")
qty_out.plot_k_level(0)
print("Executing `copy_stencil` with origin=(2, 2, 0), domain=(2, 2, nz)")
copy_stencil(qty_in, qty_out, origin=(2, 2, 0), domain=(2, 2, nz))
print(
    "Plotting qty_out at K = 0 based on `copy_stencil` with origin=(2, 2, 0), domain=(2, 2, nz)"
)
qty_out.plot_k_level(0)

### **`FORWARD` and `BACKWARD` `computation` keywords and Offset Indexing within a stencil call**

Besides `PARALLEL`, the developer can specify `FORWARD` or `BACKWARD` as the iteration policy in `K` for a stencil.  Essentially, the `FORWARD` policy has `K` iterating consecutively starting from the lowest vertical index to the highest, while the `BACKWARD` policy performs the reverse.

An array-based stencil variable can also have an integer dimensional offset if the array variable is on the right hand side of the `=` for the computation.  When a computation is performed at a particular point, an offset variable's coordinate is based on that particular point plus (or minus) the offset in the offset dimension.

The following examples demonstrate the use of these two iteration policies and also offset indexing in the `K` dimension.  Note that offsets can also be applied to the `I` or `J` dimension.

In [ ]:
from ndsl.dsl.gt4py import FORWARD, BACKWARD

nx = 5
ny = 5
nz = 5
nhalo = 1
backend = "numpy"

shape = (nx + 2 * nhalo, ny + 2 * nhalo, nz)

qty_out = Quantity(
    data=np.zeros(shape), dims=["I", "J", "K"], units="m", gt4py_backend=backend
)

arr = np.indices(shape, dtype=float).sum(
    axis=0
)  # Value of each entry is sum of the I and J index at each point
qty_in = Quantity(data=arr, dims=["I", "J", "K"], units="m", gt4py_backend=backend)

print("Plotting values of qty_in at K = 0")
qty_in.plot_k_level(0)
print("Plotting values of qty_in at K = 1")
qty_in.plot_k_level(1)
print("Plotting values of qty_in at K = 2")
qty_in.plot_k_level(2)


@stencil(backend=backend)
def mult_upward(qty_in: FloatField, qty_out: FloatField):
    with computation(FORWARD), interval(...):
        qty_out = qty_in[0, 0, -1] * 2.0


print("Executing 'mult_upward' with origin=(nhalo, nhalo, 1), domain=(nx, ny, 2)")
mult_upward(qty_in, qty_out, origin=(nhalo, nhalo, 1), domain=(nx, ny, 2))
print(
    "Plotting values of qty_out at K = 0 with origin=(nhalo, nhalo, 1), domain=(nx, ny, 2)"
)
qty_out.plot_k_level(0)
print(
    "Plotting values of qty_out at K = 1 with origin=(nhalo, nhalo, 1), domain=(nx, ny, 2)"
)
qty_out.plot_k_level(1)
print(
    "Plotting values of qty_out at K = 2 with origin=(nhalo, nhalo, 1), domain=(nx, ny, 2)"
)
qty_out.plot_k_level(2)
print(
    "Plotting values of qty_out at K = 3 with origin=(nhalo, nhalo, 1), domain=(nx, ny, 2)"
)
qty_out.plot_k_level(3)


@stencil(backend=backend)
def copy_downward(qty_in: FloatField, qty_out: FloatField):
    with computation(BACKWARD), interval(...):
        qty_out = qty_in[0, 0, 1]


print("Resetting qty_out to zeros")
qty_out = Quantity(
    data=np.zeros(shape), dims=["I", "J", "K"], units="m", gt4py_backend=backend
)

print("Executing 'copy_downward' with origin=(1, 1, 0), domain=(nx, ny, nz-1)")
copy_downward(qty_in, qty_out, origin=(1, 1, 0), domain=(nx, ny, nz - 1))
print("***")
print("Plotting values of qty_out at K = 0")
qty_out.plot_k_level(0)
print("Plotting values of qty_out at K = 1")
qty_out.plot_k_level(1)
print("Plotting values of qty_out at K = 2")
qty_out.plot_k_level(2)

Regarding offsets, GT4Py does not allow offsets to variables in the left hand side of the `=`.  Uncomment and execute the below code to see the error `Assignment to non-zero offsets is not supported.`.

In [ ]:
# @stencil(backend=backend)
# def mult_upward_error(qty_in: FloatField, qty_out: FloatField):
#     with computation(FORWARD), interval(...):
#         qty_out[0,-1,-1] = qty_in * 2.0

GTScriptSyntaxError: Assignment to non-zero offsets is not supported.

### **Limits to offset : Cannot set offset outside of usable domain**

Note that there are limits to the offsets that can be applied in the stencil.  An error will result if the specified shift results attemps to read data that is not available or allocated.  In the example below, a shift of -2 in the `J` axis will shift `field_in` out of its possible range in `J`.

In [ ]:
nx = 5
ny = 5
nz = 5
nhalo = 1
backend = "numpy"

shape = (nx + 2 * nhalo, ny + 2 * nhalo, nz)

qty_out = Quantity(
    data=np.zeros(shape), dims=["I", "J", "K"], units="m", gt4py_backend=backend
)

arr = np.indices(shape, dtype=float).sum(
    axis=0
)  # Value of each entry is sum of the I and J index at each point
qty_in = Quantity(data=arr, dims=["I", "J", "K"], units="m", gt4py_backend=backend)


@stencil(backend=backend)
def copy_stencil_offset(field_in: FloatField, field_out: FloatField):
    with computation(PARALLEL), interval(...):
        field_out = field_in[0, -2, 0]


print("Executing 'copy_stencil' with origin=(nhalo, nhalo, 0), domain=(nx, ny, nz)")
copy_stencil(qty_in, qty_out, origin=(nhalo, nhalo, 0), domain=(nx, ny, nz))
print("Executing 'copy_stencil' where qty_out is copied back to qty_in")
copy_stencil(qty_out, qty_in)
qty_in.plot_k_level(0)
print(
    "Executing 'copy_stencil_offset' where origin=(nhalo, nhalo, 0), domain=(nx, ny, nz)"
)
copy_stencil_offset(qty_in, qty_out, origin=(nhalo, nhalo, 0), domain=(nx, ny, nz))
qty_out.plot_k_level(0)

### **`if/else` statements**

GT4Py allows for `if/else` statements to exist within a stencil.  The following simple example shows a stencil `stencil_if_zero` modifing values of `in_out_field` depending on its initial value.

In [ ]:
qty_out = Quantity(
    data=np.zeros(shape), dims=["I", "J", "K"], units="m", gt4py_backend=backend
)

arr = np.indices(shape, dtype=float).sum(
    axis=0
)  # Value of each entry is sum of the I and J index at each point
qty_in = Quantity(data=arr, dims=["I", "J", "K"], units="m", gt4py_backend=backend)

print("Plotting values of qty_in at K = 0")
qty_in.plot_k_level(0)
print("Plotting values of qty_out at K = 0")
qty_out.plot_k_level(0)
print("Running copy_stencil with origin=(nhalo, nhalo, 0), domain=(nx, ny, 5)")
copy_stencil(qty_in, qty_out, origin=(nhalo, nhalo, 0), domain=(nx, ny, 5))
print(
    "Plotting values of qty_out at K = 0 based on running copy_stencil with origin=(nhalo, nhalo, 0), domain=(nx, ny, 5)"
)
qty_out.plot_k_level(0)
print(
    "Plotting values of qty_out at K = 1 based on running copy_stencil with origin=(nhalo, nhalo, 0), domain=(nx, ny, 5)"
)
qty_out.plot_k_level(1)


@stencil(backend=backend)
def stencil_if_zero(in_out_field: FloatField):
    with computation(PARALLEL), interval(...):
        if in_out_field == 0.0:
            in_out_field = 30
        else:
            in_out_field = 10


print("Running 'stencil_if_zero' on qty_out")
stencil_if_zero(qty_out)
print("Plotting values of qty_out at K = 0 based on running stencil_if_zero")
qty_out.plot_k_level(0)
print("Plotting values of qty_out at K = 1 based on running stencil_if_zero")
qty_out.plot_k_level(1)

### **Function calls**

GT4Py also has the capability to create functions in order to better organize code.  The main difference between a GT4Py function call and a GT4Py stencil is that a function does not (and cannot) contain the keywords `computation` and `interval`.  However, array index referencing within a GT4py function is the same as in a GT4Py stencil.

GT4Py functions can be created by using the decorator `function` (Note: `function` originates from the package `ndsl.dsl.gt4py`).

In [14]:
from ndsl.dsl.gt4py import function


@function
def plus_one(field: FloatField):
    return field[0, 0, 0] + 1


@stencil(backend=backend)
def field_plus_one(source: FloatField, target: FloatField):
    with computation(PARALLEL), interval(...):
        target = plus_one(source)


nx = 5
ny = 5
nz = 5
nhalo = 1
backend = "numpy"

shape = (nx + 2 * nhalo, ny + 2 * nhalo, nz)

qty_out = Quantity(
    data=np.zeros(shape), dims=["I", "J", "K"], units="m", gt4py_backend=backend
)

arr = np.indices(shape, dtype=float).sum(
    axis=0
)  # Value of each entry is sum of the I and J index at each point
qty_in = Quantity(data=arr, dims=["I", "J", "K"], units="m", gt4py_backend=backend)

print("Plotting values of qty_in at K = 0")
qty_in.plot_k_level(0)
print("Plotting values of qty_out at K = 0")
qty_out.plot_k_level(0)
print("Executing 'field_plus_one'")
field_plus_one(qty_in, qty_out)
print("Plotting values of qty_out at K = 0 after executing 'field_plus_one'")
qty_out.plot_k_level(0)